In [2]:
import pandas as pd
import numpy as np
print(pd.__version__)
print(np.__version__)

# read dataframe
df_specific = pd.read_csv("./data_wrangled/df_specific_wrangle.csv", encoding="utf-8")

# pandas 설정: 각 column 별 (혹은 한 cell에서) 출력되는 글자수 제한을 없애기
pd.set_option('display.max_colwidth', None)

1.0.5
1.19.0


In [3]:
import re
# removing junks from the string data
def remove_junk(str_data):
    str_data = str_data.replace("\\\\n", "")
    str_data = str_data.replace("\\n", "")
    str_data = str_data.replace("\n", "")
    str_data = str_data.replace("\\", "")
    str_data = str_data.replace("\\t", "")
    str_data = str_data.replace("NaN", "")

    # print(str_data) makes an error. too much of data
    # 한글, english 빼고 전부 날리기
    # str_data = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z|0-9|.,]', ' ', str_data)
    
    
    str_data = re.sub('[^가-힣ㄱ-ㅎㅏ]', ' ', str_data)

    # replace multiple spaces into single space
    str_data = ' '.join(str_data.split())
    return str_data

In [4]:
def access_univ_info(dataframe, column, univ_code):
    df_row = dataframe.loc[dataframe["대학코드"] == univ_code]
    str_univ_info = df_row[column].to_string(index=False).lstrip()
    str_univ_info = remove_junk(str_univ_info)
    return str_univ_info

In [5]:
# collecting all column values into one string
def column_to_string(dataframe, column_name: str):
    str_corpus = dataframe[column_name].to_string(index=False).lstrip()
    str_corpus = remove_junk(str_corpus)
    return str_corpus

In [13]:
info_list = ['gen_info', 'env_info', 'food_info', 'study_info', 'office_info', 'facil_info', 'mhct_info', 'help_info', 'etc_info']

In [34]:
for info in info_list:
    corpus_sample = column_to_string(df_specific, info)
    print(info, len(corpus_sample))

gen_info 3985044
env_info 3316342
food_info 4212609
study_info 3970047
office_info 2022400
facil_info 2832921
mhct_info 2307935
help_info 1582255
etc_info 2521724


## Tokenizing Sample Corpus

In [14]:
# https://github.com/lovit/soynlp
from soynlp.tokenizer import RegexTokenizer, LTokenizer

tokenizer = LTokenizer()
tokenizer

In [15]:
from collections import Counter

In [16]:
import random

In [17]:
stopwords = []

In [21]:
stopwords_candidates = []

for info in info_list:
    # info column series
    df_info = df_specific[info].fillna("")
    print(df_info.shape)
    
    # info column converted into one string
    corpus = column_to_string(df_specific, info)
    tokenized_corpus = tokenizer.tokenize(corpus)
    print(tokenized_corpus[:10])
    print(f"number of tokenized words in a corpus are: {len(tokenized_corpus)}")

    tokenized_corpus_counted = Counter(tokenized_corpus)
    print("number of counted words:", len(tokenized_corpus_counted))

    common_words = tokenized_corpus_counted.most_common(300)
    common_words_list, frequency_list = map(list, zip(*common_words))
    print(common_words_list[:10])    

    stopwords_candidates += common_words_list
    print(len(stopwords_candidates))

    for i in stopwords_candidates:
        if df_info.str.contains(i).sum() < 120:
            print(f"{i} word not appear in less than 1/4 of universities")
            stopwords_candidates.remove(i)
    
    print(len(stopwords_candidates))
    print(f"{random.sample(stopwords_candidates,10)} are random stopwords candidates")

ess than 1/4 of universities
즐길 word not appear in less than 1/4 of universities
헬스장 word not appear in less than 1/4 of universities
동아리도 word not appear in less than 1/4 of universities
추천합니다 word not appear in less than 1/4 of universities
학교의 word not appear in less than 1/4 of universities
프로그램을 word not appear in less than 1/4 of universities
체육관이 word not appear in less than 1/4 of universities
체육관 word not appear in less than 1/4 of universities
참여할 word not appear in less than 1/4 of universities
기회가 word not appear in less than 1/4 of universities
스쿼시 word not appear in less than 1/4 of universities
있어요 word not appear in less than 1/4 of universities
교내 word not appear in less than 1/4 of universities
수영 word not appear in less than 1/4 of universities
등등 word not appear in less than 1/4 of universities
가격에 word not appear in less than 1/4 of universities
헬스 word not appear in less than 1/4 of universities
외에도 word not appear in less than 1/4 of universities
행사를 word not app

In [22]:
stopwords = stopwords_candidates

In [23]:
with open("file.txt", "wt", encoding="UTF8") as output:
    output.write(str(stopwords))

## Tokenizing one column of university dataframe with soynlp

In [58]:
column_data = "gen_info"
# df_column = df_specific[df_specific[column_data].notnull()]
df_column = df_specific[column_data].fillna("")
df_process = df_column.apply(remove_junk)
%time df_tokens = df_process.apply(tokenizer.tokenize)
print(df_tokens.shape)

CPU times: user 2.03 s, sys: 5.45 ms, total: 2.03 s
Wall time: 2.03 s
(470,)


In [59]:
# make bag of words out of one university's students' reviews
import random
random_int = random.randint(0,469)
sample_univ_bow = df_tokens[random_int]
print(len(sample_univ_bow))
while len(sample_univ_bow) < 500:
    random_int = random.randint(0,469)
    sample_univ_bow = df_tokens[random_int]
    print(len(sample_univ_bow))
print(df_specific["대학명"][random_int])
sample_univ_bow[:10]

88
333
5480
ISC PARIS - School of Management


['학교는', '아주', '작습니다', '학교라기보다는', '사무실', '건물같은', '크기입니다', '그나마', '파리', '구와']

In [69]:
subtracted_bow_list = [item for item in sample_univ_bow if item not in stopwords]

In [72]:
counted_bow_keywords = Counter(subtracted_bow_list)
sample_common_words = counted_bow_keywords.most_common(100)
print(sample_common_words)

[('파리', 89), ('건물', 35), ('파리의', 35), ('위치하고', 24), ('학교는', 22), ('날씨가', 19), ('건물로', 19), ('건물은', 19), ('크지', 19), ('비가', 17), ('개의', 16), ('파리는', 16), ('수업이', 16), ('이루어져', 15), ('버스를', 14), ('기후는', 13), ('구', 13), ('날씨는', 13), ('워낙', 13), ('크기는', 13), ('외곽에', 13), ('구에', 12), ('걸어서', 12), ('역에서', 12), ('건물이', 12), ('학교의', 12), ('경계에', 12), ('총', 12), ('작습니다', 11), ('구와', 11), ('위치해', 11), ('학교가', 11), ('자체가', 11), ('가까운', 10), ('캠퍼스가', 10), ('분정도', 10), ('수업은', 9), ('겨울에는', 9), ('위치는', 9), ('기차를', 9), ('존', 9), ('타면', 9), ('떨어져', 9), ('건물에서', 9), ('않았습니다', 9), ('빌딩', 9), ('빌딩은', 9), ('아닙니다', 8), ('캠퍼스', 8), ('거리에', 8), ('파리가', 8), ('버스', 8), ('겨울', 8), ('동아리', 8), ('정도의', 8), ('거리가', 7), ('해가', 7), ('내려서', 7), ('파리에', 7), ('프랑스', 7), ('나머지', 7), ('버스로', 7), ('규모가', 7), ('까지', 7), ('세개의', 7), ('않기', 7), ('해도', 7), ('없고', 6), ('겨울은', 6), ('시내', 6), ('호선', 6), ('있어요', 6), ('캠퍼스를', 6), ('보통', 6), ('크기가', 6), ('않지만', 6), ('구성되어', 6), ('한국보다', 6), ('맑은', 6), ('여름에는', 6), ('층', 6), ('우산을',

In [73]:
# zipping deletes overlapping string items
zipped_common_words, frequency_list = map(list, zip(*sample_common_words))
print(zipped_common_words)

['파리', '건물', '파리의', '위치하고', '학교는', '날씨가', '건물로', '건물은', '크지', '비가', '개의', '파리는', '수업이', '이루어져', '버스를', '기후는', '구', '날씨는', '워낙', '크기는', '외곽에', '구에', '걸어서', '역에서', '건물이', '학교의', '경계에', '총', '작습니다', '구와', '위치해', '학교가', '자체가', '가까운', '캠퍼스가', '분정도', '수업은', '겨울에는', '위치는', '기차를', '존', '타면', '떨어져', '건물에서', '않았습니다', '빌딩', '빌딩은', '아닙니다', '캠퍼스', '거리에', '파리가', '버스', '겨울', '동아리', '정도의', '거리가', '해가', '내려서', '파리에', '프랑스', '나머지', '버스로', '규모가', '까지', '세개의', '않기', '해도', '없고', '겨울은', '시내', '호선', '있어요', '캠퍼스를', '보통', '크기가', '않지만', '구성되어', '한국보다', '맑은', '여름에는', '층', '우산을', '그나마', '기차', '봄', '버스가', '생각하면', '구의', '행정구역상', '지하철', '않았던', '날씨', '동으로', '모여', '끝', '학교입니다', '서울보다', '생각하시면', '때도', '학년']


### Keywords accordign to Sentences order

In [62]:
counted_bow = Counter(sample_univ_bow)
counted_bow_tuple_list = list(Counter(counted_bow).items())
print(counted_bow_tuple_list[:50])

[('학교는', 22), ('아주', 6), ('작습니다', 11), ('학교라기보다는', 3), ('사무실', 3), ('건물같은', 1), ('크기입니다', 3), ('그나마', 5), ('파리', 89), ('구와', 11), ('가까이', 3), ('있기는', 3), ('하지만', 25), ('엄밀히', 1), ('말하면', 2), ('파리지역은', 1), ('아닙니다', 8), ('파리보다', 1), ('조금', 11), ('북쪽인', 1), ('외곽지역에', 1), ('해당됩니다', 1), ('그래서', 7), ('치안이', 1), ('썩', 2), ('좋은', 7), ('편은', 2), ('저는', 10), ('학교에서', 7), ('좀', 7), ('거리가', 7), ('있는', 34), ('기차역이나', 1), ('버스정류장을', 1), ('주로', 16), ('이용했었어서', 1), ('다리밑을', 1), ('지나는데', 1), ('노숙자있고', 1), ('으스스해서', 1), ('분위기가', 3), ('쎄', 1), ('했습니다', 2), ('가까운', 10), ('기차역이', 2), ('있는데', 13), ('거기로', 1), ('통학하신다면', 1), ('큰', 13), ('문제는', 1)]


In [63]:
# zipping deletes overlapping string items
zipped_bow_words, bow_frequency_list = map(list, zip(*counted_bow_tuple_list))
zipped_bow_words[:10]

['학교는', '아주', '작습니다', '학교라기보다는', '사무실', '건물같은', '크기입니다', '그나마', '파리', '구와']

In [64]:
zipped_bow_words_subtracted = [item for item in zipped_bow_words if item not in stopwords]

In [65]:
print(len(zipped_bow_words_subtracted))

2244


In [66]:
print(random.sample(zipped_bow_words_subtracted, 50))

['다녀왔는데', '걸어가면', '분야만', '일주일이나', '바이지만', '작고', '맞고', '졸업반', '아니므로', '날의', '날씨이며', '동네가', '다녀와서', '겨울이었습니다', '들려볼', '가까이에', '도시였다', '학원', '놓고', '힘들', '캠퍼스다', '중심지와는', '땐', '말해주신', '굴다리에', '라자르', '멀리에', '계속되어', '영미권으로', '느끼지는', '공', '외관을', '분야별로', '있다고는', '처음엔', '교내의', '호주', '가장자리에', '않는데', '조금더', '남쪽의', '월에', '동네는', '구부터', '지하철에서', '안에서', '한인마트', '몇개가', '어려움을', '어려웠습니다']


In [74]:
summary = " ".join(zipped_bow_words_subtracted)
summary

'학교는 작습니다 학교라기보다는 사무실 건물같은 크기입니다 그나마 파리 구와 가까이 있기는 엄밀히 말하면 파리지역은 아닙니다 파리보다 북쪽인 외곽지역에 해당됩니다 치안이 썩 편은 거리가 기차역이나 버스정류장을 이용했었어서 다리밑을 지나는데 노숙자있고 으스스해서 분위기가 쎄 가까운 기차역이 거기로 통학하신다면 문제는 없으시겠지만 저처럼 타야하는 오후 수업은 시간표에 넣는걸 추천드리지 않아요 구에 거주하신다면 걸어서 정도일 가깝습니다 파업때는 걸어다니는 친구도 있었어요 겨울에는 해가 반에 지기 시작합니다 초에는 늦게도 떠있어서 좋았는데 막판에는 교통파업에 날씨도 어두컴컴해서 좋게 지내다 오지는 못한 정확히 옆 동네 르발루아 페레에 모르고 한복판에 숙소를 구했다가 원래 걸리는 거리를 파업 반씩 걸려서 다녔습니다 아닐 다닐만한 거리였습니다 자체는 오르막 내리막 없고 다니는 길이 좁으며 겨울은 비가 내리고 춥습니다 시내 끝자락에 위치해 구인데 붙어있는 수준입니다 호선 역에서 내려서 걸어가면 공사장 주변이고 지나야 굴다리에 노숙자들이 많아서 주변에 집을 구한 아니고 시내나 남부 유학생들은 거주합니다 살아서 지하철을 이용해야 한다면 생 라자르 선을 내리는 존이지만 학교까지 분도 걸립니다 대신 표를 낱장으로 사면 열차는 기차 개념이라서 티켓을 구매해야 해요 나비고나 이마지네 카드 사셔서 추가요금 등교하세요 엄청 작아요 건물이 개라는데 교환학생들의 개의 건물에서만 진행되고 건물들은 밥 먹으러 식당 가거나 국제처에 서류 받으러 가볼 때만 파리는 우리나라의 대학과 캠퍼스가 있지 대학들이 건물 몇개로 이루어져있죠 그렇기 광활하고 푸릇푸릇한 캠퍼스 기대하신다면 파리보다는 스위스나 호주 영미권으로 추천드려요 파리에서도 북쪽에 위치해있어요 통학을 했었고 옆에 마트가 있어요 거기서 친구들이랑 점심 사먹고 쇼핑도 그랬어요 기후는 가을학기보다는 봄학기에 좋으실 겨울에 낮이 무척 짧고 다섯시만 되어도 깜깜해져요 하늘도 칙칙해서 겨울보다는 길고 하늘이 예쁜 여름이 좋을거에요 봄 오시면 금방 월부

In [ ]:
summary